 # Libraries and Data

In [1]:
from zipfile import ZipFile

import altair as alt
import numpy as np
import pandas as pd


 After downloading the dataset, we can load the train set from the zip file.

 However, for faster reloads, we stored it in a feather file.

 You can comment/uncomment the specific snippets for each use case.

In [2]:
# Original data load
zipfile = ZipFile("./data/avazu-ctr-prediction.zip")
train = pd.read_csv(
    zipfile.open("train.gz"), compression="gzip", usecols=["click", "hour"]
)

In [3]:
# Save to feather, for faster data reloads
train.to_feather("./data/train.feather")
assert train.equals(pd.read_feather("./data/train.feather"))

In [4]:
# Load from feather
train = pd.read_feather("./data/train.feather")


In [5]:
# Transforming the datetimeformat to pandas datetime

train["dthour"] = pd.to_datetime(train["hour"], format="%y%m%d%H")
assert (
    train["hour"].astype(str).str[-2:].astype(int) == train["dthour"].dt.hour
).all(), "Hour transformation do not match"

train = train.set_index("dthour").drop(columns="hour")

In [6]:
assert all(
    pd.Series(train.index).diff().iloc[1:].dt.total_seconds().unique() / 60**2
    == [
        0,
        1,
    ]
), f"Incorrect timestamp deltas"

assert train.index.is_monotonic, "Timestamp is not monotonic."


 # Data aggregation

 We assume that that it is meaningful to use all the ads in a single,
 group and to plot them all onto the same time series.

 We assume that a row in the dataset stands for an 'impression' and,
 therefore, we can get hourly CTRs by dividing the number of clicks with
 the number of total impressions within that hour.



In [7]:

hourly = pd.DataFrame()
hourly["CTR"] = train.resample("H")["click"].mean()
hourly["clicks"] = train.resample("H")["click"].sum()
hourly["impressions"] = train.resample("H")["click"].count()


In [8]:
line = alt.Chart(hourly.reset_index()).mark_line().encode(x="dthour:T", y="CTR:Q")

points = (
    alt.Chart(hourly.reset_index())
    .mark_point()
    .encode(x="dthour:T", y="CTR:Q", tooltip=["dthour", "CTR"])
)

(line + points).properties(title="CTR", width=600, height=150)


alt.LayerChart(...)

 # Outlier detection

 As the data contains only a single weekend, we cannot tell too much about the weekly
 patterns.


 ## Assumptions

 - We do this for retrospective analysis, and therefore we can use a
 centered moving window
 - We do not have a specific use case, so we can experiment with different
 window sizes. For in-day outliers we can set it to 6H, while for in-week
 or in-month outliers we can set it to 3D, 7D, etc.

 ## Calculation

 Because CTR is already an aggregate metrics, we need to calculate the
 rolling metrics from the original `clicks` column.

In [9]:


def rolling_metrics(hourly, window):

    try:
        hourly.set_index("dthour", inplace=True)
    except KeyError:
        print("`dthour` is already an index")

    hourly[f"{window}-mean"] = (
        hourly.rolling(window, center=True)["clicks", "impressions"]
        .sum()
        .apply(lambda x: x["clicks"] / x["impressions"], axis=1)
    )

    hourly["squared_error"] = (hourly["CTR"] - hourly[f"{window}-mean"]) ** 2
    hourly[f"{window}-squared_error"] = (
        hourly["squared_error"].rolling(window, center=True).sum()
    )

    hourly["hours_in_window"] = (
        hourly.rolling(window, center=True).apply(lambda x: x.size).iloc[:, 0]
    )
    hourly[f"{window}-std"] = np.sqrt(
        hourly[f"{window}-squared_error"] / hourly["hours_in_window"]
    )

    return hourly



In [10]:
def define_outliers(hourly, window):
    hourly["top"] = hourly[f"{window}-mean"] + hourly[f"{window}-std"] * 1.5
    hourly["bottom"] = hourly[f"{window}-mean"] - hourly[f"{window}-std"] * 1.5
    hourly["outlier"] = (hourly["CTR"] > hourly["top"]) | (
        hourly["CTR"] < hourly["bottom"]
    ).astype(bool)
    return hourly



In [11]:
def plot_outliers(hourly):
    try:
        hourly.reset_index("dthour", inplace=True)
    except KeyError:
        print("`dthour` is already a column")

    points = (
        alt.Chart(hourly)
        .mark_point()
        .encode(
            x="dthour:T",
            y=alt.Y("CTR:Q", scale=alt.Scale(zero=False)),
            color=alt.Color("outlier:N"),
            tooltip=["dthour:T", "CTR", "outlier"],
        )
    )

    lines = alt.layer(
        alt.Chart(hourly)
        .mark_line(opacity=0.5, color="grey")
        .encode(x="dthour:T", y="CTR:Q"),
        alt.Chart(hourly)
        .mark_line(opacity=0.5, color="red")
        .encode(x="dthour:T", y=f"{window}-mean:Q"),
        alt.Chart(hourly)
        .mark_area(opacity=0.2)
        .encode(x="dthour:T", y="top:Q", y2="bottom:Q"),
    )

    (points + lines).properties(title="CTR Outliers", width=600, height=150).display()



In [12]:
window = "12H"

hourly = rolling_metrics(hourly, window)
hourly = define_outliers(hourly, window)

print(hourly["outlier"].astype(int).describe())
display(
    hourly.loc[
        :, ["CTR", f"{window}-mean", f"{window}-std", "top", "bottom", "outlier"]
    ]
)

plot_outliers(hourly)


`dthour` is already an index
count    240.000000
mean       0.120833
std        0.326615
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: outlier, dtype: float64


,CTR,12H-mean,12H-std,top,bottom,outlier
dthour,,,,,,
2014-10-21 00:00:00,0.174714,0.158968,0.012419,0.177597,0.140340,False
2014-10-21 01:00:00,0.173695,0.160341,0.011625,0.177778,0.142904,False
2014-10-21 02:00:00,0.150696,0.161416,0.010988,0.177898,0.144933,False
2014-10-21 03:00:00,0.169791,0.161669,0.010922,0.178052,0.145287,False
2014-10-21 04:00:00,0.151206,0.163139,0.010414,0.178761,0.147518,False
...,...,...,...,...,...,...
2014-10-30 19:00:00,0.194021,0.181820,0.010248,0.197192,0.166448,False
2014-10-30 20:00:00,0.187967,0.184328,0.010416,0.199951,0.168704,False
2014-10-30 21:00:00,0.186728,0.185507,0.011031,0.202054,0.168960,False


alt.LayerChart(...)